# Comparing Models with Bayesian Evidence and Information Criteria

## Introduction

We'll be looking at some fake but realistic data and comparing how well different models do at describing it.

It's a simple data set, just a list of numbers. Each one represents a measured distance between two different estimates of the center of a galaxy cluster: the location of the brightest galaxy and a centroid of the emissive, diffuse gas. The context here is that automated algorithms sometimes fail to chose the central galaxy correctly (because of image artifacts or other problems), whereas the gas centroid is more reliable but also more expensive to measure. Therefore, we'd like to use this data set to characterize the distribution of mis-centerings so that the galaxy-based centers can be used for large sample, with the resulting errors propagated forward through future processing, e.g., weak lensing estimates.

Let's load up the data and have a look.

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

plt.rcParams['xtick.labelsize'] = 'large'
plt.rcParams['ytick.labelsize'] = 'large'
%matplotlib inline
# or %matplotlib notebook

In [ ]:
y = np.loadtxt('model_evaluation.dat')

In [ ]:
plt.rcParams['figure.figsize'] = (6.0, 4.0)
plt.hist(y, bins=50);

## Problem 1: evaluating a simple model

First, let's fit and evaluate a simple model for the data. I propose the exponential distribution, but feel free to use a different guess.

**Model 1:** $P(y|x_1) = \frac{1}{x_1}e^{-y/x_1}$; $y\geq0$

#### 1a) Select a prior distribution for Model 1.

Make sure it's a proper (normalizable) distribution. We don't want to deal with improper distributions when calculating the evidence later on.

For e.g., to adopt a uniform prior, you could just use statements like the ones below, with appropriate implementation of later functions:

In [ ]:
x1_lo =  # lower bound
x1_hi =  # upper bound

#### 1b) Implement likelihood, posterior, and predictive functions for Model 1.

This code should get you started.

In [ ]:
# Log-likelihood function
def lli1(x1):
    # uses global 'y' variable where the data are stored
    # x1 is a scalar
    return np.sum(st.expon.logpdf( # complete

# For convenience. This function can be called with a vector argument, unlike lli1.
loglike1 = np.vectorize(lli1)

# Log-posterior function. We'll use this with an MCMC sampler, so it should call the non-vectorized likelihood.
def post1(p):
    x1 = p[0]
    # something to do with the prior here, setting the value of pr
    like = lli1(x1)
    return like + pr

# Posterior-predictive density
def postpredpdf1(yy, x1):
    return st.expon.pdf( # complete

# Perform posterior prediction
def postpred1(n, x1):
    # return n mock data points from the model
    return st.expon.rvs( # complete

#### 1c) Fit Model 1.

Code below will do parameter inference, look at the resulting Markov chains, remove burn-in, thin, and concatenate chains. Since this step isn't really the point of this problem, it's mostly given, but you'll still need to fill in a few pieces. Make sure you understand the format that the MCMC results are stored in (e.g. the `samples1` variable).

In [ ]:
import emcee

Set up the sampling problem.

In [ ]:
# example sampler setup
nwalkers = 100
ndim = 1 # number of parameters
lnprob = post1
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob) # add e.g. threads=4 to speed things up with multiprocessing

Define starting points for each chain in the ensemble. Given a guess at the right answer, this code initializes them to be at the guess $\pm1$%. For later, note that this array should have dimensions `Nwalkers` $\times$ `Nparameters`.

In [ ]:
guess = # complete
theta1_0 = np.array([[guess]*(1.0 + 0.01*np.random.rand(ndim)) for j in range(nwalkers)])
theta1_0.shape

Run the sampler and look at the resulting chains.

In [ ]:
sampler.run_mcmc(theta1_0, 1000)
ens1 = sampler.chain
plt.rcParams['figure.figsize'] = (12.0, 3.0)
for j in range(nwalkers): plt.plot(ens1[j,:,0], 'o');

Remove obvious burn-in and do some thinning.

In [ ]:
burn = # complete
thinby = # complete
samples1 = ens1[:, burn:, :].reshape((-1, ndim))
samples1 = samples1[range(0,samples1.shape[0],thinby),:]
samples1.shape

It will be useful for later to know the mean of the posterior.

In [ ]:
postmean1 = np.mean(samples1[:,0])
postmean1

#### 1d) Visually compare the predictions of Model 1 with the data.

First, let's just plot the posterior-mean model over the data.

In [ ]:
plt.rcParams['figure.figsize'] = (6.0, 4.0)
plt.hist(y, bins=50, normed=True)
yy = np.arange(1000.)
plt.plot(yy, postpredpdf1( # complete

Now, let's compare a random predicted data set from the posterior with the data.

In [ ]:
j = # arbitrary posterior sample
mock = postpred1( # complete
plt.hist(y, bins=50)
plt.hist(mock, bins=50, color=[1,0,0,0.5]);

Qualitatively, would you say the model is a good fit?

#### 1e) Choose a simple test statistic and use it to compare the data with the posterior predictive distribution.

Do this visually, as well as calculating the corresponding $p$-value.

In [ ]:
# example setup is for a test function of data only (no parameter dependence)
def T(yy):
    return # complete
# compute T from draws from every posterior sample
pp = np.array([T(postpred1(len(y), x1)) for x1 in samples1[:,0]])

In [ ]:
# show a histogram of pp
# compare that distribution with T(real data)
# compute and report the p-value for T

Does the quantitative goodness-of-fit match your expectation?

#### 1f) Calculate the DIC for Model 1.

Also compute the value of $p_D$. (Does it make sense?) We'll compare these to the corresponding values for a different model later.

In [ ]:
# compute things
pD1 = # complete
DIC1 = # complete
print pD1, DIC1

#### 1g) Compute the evidence for Model 1.

To do this, note that

$P(D|H)=\int P(D|\theta,H) \, P(\theta|H) d\theta$

can be approximated by an average over samples from the prior

$P(D|H) \approx \sum_{k=1}^m P(D|\theta_k,H)$; $\theta_k\sim P(\theta|H)$.

This approach is not especially efficient or numerically stable in general (because the likelihood typically is large in only a small fraction of the prior volume), but it will do for this exercise.

Draw a large number of samples ($m$) from the prior and use them to calculate the evidence. In fact, to avoid numerical over/underflows, we'll need to add a constant to the log-likelihoods before exponentiating them in the average (which of course means that we don't actually get the evidence out! This is another reason to prefer better methods that more stably produce the evidence or log-evidence). To compare evidences from 2 models, we'll of course need to add the *same* constant in each case. Nevertheless, it's worth putting together the machinery to do these calculations now.

In [ ]:
nmc = # suitable large number of draws from the prior
x1_from_prior = # complete
ll_from_prior1 = loglike1( # complete

ev1 = # complete
print ev1
    
# NB do not compare ev1 to the evidence for another model unless the same constant has been used to offset the
# two log-likelihoods!

## Problem 2: evaluating a more complex model

Choose a more complex model to compare to Model 1. I suggest incorporating another standard PDF (implemented in `scipy`) into the likelihood, with a parameter indicating the relative proportion of the two components, such that

$P(y|\theta_1,\theta_2,f) = f\,P_1(y|\theta_1) + (1-f)\,P_2(y|\theta_2)$.

But feel free to do something different if you like.

Next, repeat the various fitting and evaluating steps that we did for Model 1 for this new model.

#### 2a) Specify Model 2, and a choose sensible prior distribution.

If following the hint above, don't forget that $f$ is also a parameter of the model!

#### 2b) Implement likelihood, posterior, and predictive functions for Model 2.

In [ ]:
# Log-likelihood function
def lli2( # complete

# For convenience. This function can be called with a vector argument, unlike lli2.
loglike2 = np.vectorize(lli2)

# Log-posterior function. We'll use this with an MCMC sampler, so it should call the non-vectorized likelihood.
def post2(p):
    # complete

# Posterior-predictive density
def postpredpdf2(yy, # complete

# Perform posterior prediction
def postpred2(n, f, # complete
    # return n mock data points from the model
    # if your model is built out of two components with relative proportions f and (1-f), you could do something like:
    y1 = # complete - n draws from the first component
    y2 = # complete - n draws from the second component
    r = np.random.sample(n)
    return np.concatenate((y1[np.where(r <= frac)], y2[np.where(r > frac)]))

#### 2c) Fit Model 2.

The code above should still work, with relatively minor changes.

#### 2d) Visually compare the predictions of Model 2 with the data.

Do the same quantitative posterior predicive test as before, also. Does the new model fit the data better?

#### 2e) Compare the data with the posterior predictive distribution of your test statistic for Model 2.

#### 2f) Calculate the DIC and $p_D$ for Model 2.

In [ ]:
# complete
print pD2, DIC2

#### 2g) Compute the evidence for Model 2.

As before.

## Problem 3: compare models

#### 3a) Use the difference in DIC between models 1 and 2 to conclude something about the relative fitness of the two models.

In [ ]:
print DIC2 - DIC1

#### 3b) Use the evidence ratio of the two models to conclude something about their relative probability.

You'll need to re-calculate the evidences in this step. As before, the log-likelihoods will need to be offset by a constant to avoid numerical over/underflows, but of course the *same* offset needs to be applied to each model.

In [ ]:
# complete
print logevidence1, logevidence2, logevidence2-logevidence1, np.exp(logevidence2-logevidence1)

## Challenge problem

Use `emcee`'s parallel tempering functionality to calculate the log-evidences for the two models above (see the [documentation](http://dan.iel.fm/emcee/current/user/pt/)). Compare the difference in log-evidence with what you found above.

## Mega-Challenge problem

[This notebook](challenge_thermo_integ.ipynb) provides the opportunity to delve into thermodynamic integration as a method of computing the evidence (this is what `emcee` does in the challenge problem above) in more detail, to see how and why it works.